In [1]:
%matplotlib qt
import mib2hspy as m2h
import pandas as pd
import hyperspy.api as hs

from math import nan

In [2]:
parameters = m2h.MicroscopeParameters()
parameters.acceleration_voltage = 200000
parameters.cameralength = (8, nan)
parameters.camera = 'Merlin'
parameters.microscope = '2100F'
table = pd.read_excel(r'C:\Users\emilc\OneDrive - NTNU\NORTEM\Calibrations\2100F\Calibrations.xlsx', engine='openpyxl')
parameters.set_values_from_calibrationtable(table)


converter = m2h.Converter(microscope_parameters=parameters)
converter.data_path = r'C:\Users\emilc\OneDrive - NTNU\NORTEM\Merlin\2020_09_12_NanowireSC58A8_MerlinCalibrations\NWX\2020_09_12_NanowireSC58A8_Merlin_scans\20200912 132936 Scan10_2\NWX_scan1_8cm_scan10.mib'

print(converter)

No calibration found for Magnification('Magnification', nan, '', nan) in calibration table after querying for "`Nominal Magnification ()` == @nan & `Mode` == "None" & `Mag mode` == "None" & `Camera` == "Merlin" & `Acceleration Voltage (V)` == 200000 & `Microscope` == "2100F" & `Nominal Magnification ()` == @nan".

Unable to query calibration table for 
"`Nominal Rocking angle (deg)` == @nan & `Mode` == "None" & `Alpha` == @nan & `Acceleration Voltage (V)` == 200000 & `Microscope` == "2100F" & `Nominal Rocking angle (deg)` == @nan"
due to missing (required) columns. Please check that the calibration file column headers for errors. Continuing without calibrating this value.

Unable to query calibration table for 
"`Nominal Step Y (nm)` == @nan & `Mode` == "None" & `Alpha` == @nan & `Acceleration Voltage (V)` == 200000 & `Microscope` == "2100F" & `Nominal Step Y (nm)` == @nan"
due to missing (required) columns. Please check that the calibration file column headers for errors. Continuing w

In [3]:
converter.read_mib()
converter.reshape(100, 100)
converter.rechunk(32)
converter.apply_calibrations()
converter.set_metadata()
print(converter)

This mib file appears to be TEM data. The stack is returned with no reshaping.
Loaded file "C:\Users\emilc\OneDrive - NTNU\NORTEM\Merlin\2020_09_12_NanowireSC58A8_MerlinCalibrations\NWX\2020_09_12_NanowireSC58A8_Merlin_scans\20200912 132936 Scan10_2\NWX_scan1_8cm_scan10.mib" successfully:
Data: <LazyElectronDiffraction2D, title: , dimensions: (10000|256, 256)>
HDR: Content of Medipix HDR file "C:\Users\emilc\OneDrive - NTNU\NORTEM\Merlin\2020_09_12_NanowireSC58A8_MerlinCalibrations\NWX\2020_09_12_NanowireSC58A8_Merlin_scans\20200912 132936 Scan10_2\NWX_scan1_8cm_scan10.hdr":
	Time and Date Stamp (day, mnth, yr, hr, min, s): 12/09/2020 13:29:36
	Chip ID: W559_G11, - , - , -
	Chip Type (Medipix 3.0, Medipix 3.1, Medipix 3RX): Medipix 3RX
	Assembly Size (NX1, 2X2): 1x1
	Chip Mode  (SPM, CSM, CM, CSCM): SPM
	Counter Depth (number): 12
	Gain: SLGM
	Active Counters: Alternating
	Thresholds (keV): 4.000000E+1,5.110000E+2,0.000000E+0,0.000000E+0,0.000000E+0,0.000000E+0,0.000000E+0,0.000000E+0


In [6]:
converter.write('.png', height=0.01, num_frames=2)

C:\Users\emilc\Documents\mib2hspy\mib2hspy\Tools\conversion.py:473: UserWarning: <LazyElectronDiffraction2D, title: , dimensions: (100, 100|256, 256)> contains a stack of images. Converting this data to .png is not advised as it will create 10000 files
  warn(
C:\Users\emilc\Documents\mib2hspy\mib2hspy\Tools\conversion.py:396: UserWarning: Preparing plots for a stack is not advised.
  warn('Preparing plots for a stack is not advised.')
C:\Users\emilc\Documents\mib2hspy\mib2hspy\Tools\conversion.py:396: UserWarning: Preparing plots for a stack is not advised.
  warn('Preparing plots for a stack is not advised.')


In [9]:
print(frame)
print(len(frame))

<LazyElectronDiffraction2D, title: , dimensions: (|256, 256)>
1


In [10]:
min(frame.axes_manager[-2].axis)

-1.7324865161109209

In [ ]:
converter.data.plot()

C:\Users\emilc\Documents\mib2hspy\mib2hspy\Tools\conversion.py:396: UserWarning: Preparing plots for a stack is not advised.
  warn('Preparing plots for a stack is not advised.')


WriteError: 'DataAxis' object is not iterable

In [10]:
converter.data.save(str(converter.data_path.with_suffix('.hspy')), overwrite=True, chunks=converter._chunks)

In [ ]:
converter.writ

In [6]:
signal = hs.load(str(converter.data_path.with_suffix('.hspy')))

In [7]:
print(signal)
print(signal.axes_manager)
print(signal.metadata)
print(signal.original_metadata)
#signal.original_metadata

<ElectronDiffraction2D, title: , dimensions: (100, 100|256, 256)>
<Axes manager, axes: (100, 100|256, 256)>
            Name |   size |  index |  offset |   scale |  units 
================ | ====== | ====== | ======= | ======= | ====== 
     <undefined> |    100 |      0 |       0 |       1 | <undefined> 
     <undefined> |    100 |      0 |       0 |       1 | <undefined> 
---------------- | ------ | ------ | ------- | ------- | ------ 
              kx |    256 |        |    -1.7 |   0.014 | $A^{-1}$ 
              ky |    256 |        |    -1.7 |   0.014 | $A^{-1}$ 
├── Acquisition_instrument
│   └── TEM
│       ├── Detector
│       │   └── Diffraction
│       │       └── camera_length = 16.20267322524813
│       ├── beam_energy = 200000
│       ├── convergence_angle = nan
│       ├── rocking_angle = nan
│       └── rocking_frequency = nan
├── General
│   └── title = 
└── Signal
    ├── binned = False
    └── signal_type = electron_diffraction

├── Acquisition_instrument
│   └── Pa

In [7]:
min(converter.data.axes_manager[-2].axis)

-1.7324865161109209

In [ ]:
signal.plot()

In [16]:
tuple([32]*len(np.shape(converter.data)))

(32, 32, 32, 32)

In [3]:
signal = hs.load(r'C:\Users\emilc\OneDrive - NTNU\NORTEM\Calibrations\2100F\Merlin\2020_11_21_TEDPELLA673\CL8cm.hspy')
cl = m2h.get_calibration_from_MERLIN(signal)
print(cl)
cl.as_dataframe()

Cameralength 8: 16.2 cm


,Label,Nominal Cameralength (cm),Cameralength (cm),Date,Scale (1/Å),Acceleration Voltage (V),Mag mode,Camera,Microscope,Scale (mrad),Scale (deg)
0,DIFF,8.0,16.202673,2020-11-23,0.013535,200000,SAEDP,Merlin,2100F,0.33945,0.019449


In [5]:
signal = hs.load(r'C:\Users\emilc\OneDrive - NTNU\NORTEM\Calibrations\2100F\Merlin\2020_11_21_TEDPELLA673\SAMAG10k.hspy')
mag = m2h.get_calibration_from_MERLIN(signal)
mag.nominal_value=10000
mag.parameters['Mag mode']='SAMAG'
print(mag)
mag.as_dataframe()

Magnification 10000: 16830


,Label,Nominal Magnification (),Magnification (),Date,Scale (nm),Mag mode,Camera,Microscope
0,IMG,10000,16830.0,2020-11-23,3.267974,SAMAG,Merlin,2100F


In [5]:
signal.original_metadata

├── Acquisition_instrument
│   └── TEM
│       ├── acceleration_voltage = 200000.0
│       ├── alpha = Alpha 3
│       ├── mode = SAEDP
│       ├── nominal_cameralength = 8
│       └── spot = Spot 1
├── HDR
│   ├── Acquisition Type (Normal, Th_scan, Config) = Normal
│   ├── Active Counters = Counter 0
│   ├── Assembly Size (NX1, 2X2) = 1x1
│   ├── Chip ID = W559_G11, - , - , -
│   ├── Chip Mode  (SPM, CSM, CM, CSCM) = SPM
│   ├── Chip Type (Medipix 3.0, Medipix 3.1, Medipix 3RX) = Medipix 3RX
│   ├── Counter Depth (number) = 24
│   ├── DAC File = c:\MERLIN_Quad_Config\W559_G11\W559_G11_SPM.dacs,,,
│   ├── DACs = 068,511,000,000,000,000,000,000,100,255,100,125,100,100,080,100,090,030,128,004,255,126,128,174,172,511,511
│   ├── Dead Time File = Dummy (C:\<NUL>\)
│   ├── Flat Field File = None
│   ├── Frames in Acquisition (Number) = 1
│   ├── Frames per Trigger (Number) = 1
│   ├── Gain = SLGM
│   ├── Gap Fill Mode = None
│   ├── Humidity (%) = Board Humidity 0.000000
│   ├── Medipix Clo

In [ ]:
cameralengths = [
    (8, 16.2)
]

cl1 = m2h.Cameralength(8, 16.2, '2020-12-07',acceleration_voltage=200000, camera='Merlin')
#cl1.calibrate_cameralength()
cl1.calibrate_scale()
print(cl1.as_dataframe())

In [4]:
mag1 = m2h.Magnification(12000, 1, '2020-12-07', scale=0.979, acceleration_voltage=200000, camera='US1000')
#mag1.calibrate_scale()
mag1.calibrate_magnification()
print(mag1)

Magnification 12000: 14300


In [ ]:
m1 = m2h.Magnification(8000, 12030, '2020-12-07', acceleration_voltage=200000, mode='TEM', camera='Ultrascan', mag_mode='SAMAG')
cl1 = m2h.Cameralength(8, 16.2, '2020-12-07', scale=0.00134, spot_size=0.5, acceleration_voltage=200000, camera='Merlin')
cl2 = m2h.Cameralength(10, 19.3, '2020-12-07', acceleration_voltage=200000, camera='Merlin')
cl3 = m2h.Cameralength(10, 19.3, '2020-12-07', acceleration_voltage=200000, camera='Ultrascan')
step1 = m2h.StepSize(2.5, 2.4, '2020-12-07', direction='X', mode='STEM', alpha=None)
precession_calibration = m2h.PrecessionAngle(1.0, 1.04, 20., '2020-12-07', mode='NBD', alpha='Alpha 4', deflectors={'Upper_1': {'X': {'A':0.1, 'P':39}, 'Y': {'A': 0.5, 'P': 44}}})
spot1 = m2h.Spotsize(3, '2020-12-07', spot_size=0.5, mode='NBD', alpha=4, acceleration_voltage=200000)
calibrations = m2h.CalibrationList()
calibrations+=m1
calibrations+=cl1
calibrations+=cl2
calibrations+=cl3
calibrations+=step1
calibrations += precession_calibration
calibrations += spot1
#print([calibration.nominal_value for calibration in calibrations])
print(calibrations.dataframe)


In [5]:
cl1.as_dataframe()
#spot1.as_dataframe()

,Label,Nominal Cameralength (cm),Cameralength (cm),Date,Acceleration Voltage (V),Nominal Spotsize (nm),Camera,Scale (mrad),Scale (deg)
0,DIFF,8.0,16.2,2020-12-07,200000,0.5,Merlin,0.033606,0.001926


In [ ]:
new_calibrations = m2h.generate_from_dataframe(calibrations.dataframe)
print(new_calibrations.dataframe)

In [5]:
precession_calibration.as_dataframe(ignore_nans=False)

,Label,Nominal Precession Angle (deg),Precession Angle (deg),Date,Mode,Alpha,Precession excitation (%),Deflector Upper 1 X Amplitude (%),Deflector Upper 1 X Phase (deg),Deflector Upper 1 Y Amplitude (%),...,Deflector Upper 2 Y Amplitude (%),Deflector Upper 2 Y Phase (deg),Deflector Lower 1 X Amplitude (%),Deflector Lower 1 X Phase (deg),Deflector Lower 1 Y Amplitude (%),Deflector Lower 1 Y Phase (deg),Deflector Lower 2 X Amplitude (%),Deflector Lower 2 X Phase (deg),Deflector Lower 2 Y Amplitude (%),Deflector Lower 2 Y Phase (deg)
0,PREC,1.0,1.04,2020-12-07,NBD,Alpha 4,20.0,0.1,39.0,0.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
step1 = m2h.StepSize(2, 1.9, '2020-12-07', mode='STEM', alpha=None)
print(step1.as_dataframe())

In [ ]:
cl3.scale

In [ ]:
print(calibrations['`Nominal Cameralength (cm)`==8.0'])

In [ ]:

#print(precession_calibration)
print(precession_calibration.as_dataframe(ignore_nans=False))

In [ ]:
from mib2hspy import DiffractionScale

In [ ]:
cl3.scale

In [ ]:
cl3.scale

In [ ]:
print(calibrations['`Nominal Cameralength (cm)`==8.0'])

In [ ]:

#print(precession_calibration)
print(precession_calibration.as_dataframe(ignore_nans=False))

In [ ]:
from mib2hspy import DiffractionScale

In [ ]:
cl3.scale

In [ ]:
print(calibrations['`Nominal Cameralength (cm)`==8.0'])

In [2]:

#print(precession_calibration)
print(precession_calibration.as_dataframe(ignore_nans=False))

  Label  Nominal Precession Angle (deg)  Precession Angle (deg)        Date  \
0  PREC                             1.0                    1.04  2020-12-07   

  Mode    Alpha  Precession excitation (%)  Deflector Upper 1 X Amplitude (%)  \
0  NBD  Alpha 4                       20.0                                NaN   

   Deflector Upper 1 X Phase (deg)  Deflector Upper 1 Y Amplitude (%)  ...  \
0                              NaN                                NaN  ...   

   Deflector Upper 2 Y Amplitude (%)  Deflector Upper 2 Y Phase (deg)  \
0                                NaN                              NaN   

   Deflector Lower 1 X Amplitude (%)  Deflector Lower 1 X Phase (deg)  \
0                                NaN                              NaN   

   Deflector Lower 1 Y Amplitude (%)  Deflector Lower 1 Y Phase (deg)  \
0                                NaN                              NaN   

   Deflector Lower 2 X Amplitude (%)  Deflector Lower 2 X Phase (deg)  \
0      

In [ ]:
from mib2hspy import DiffractionScale

In [ ]:
cl = DiffractionScale(0.000135)
print(cl)
print(cl.to_inv_nm(200000))
print(cl.calculate_cameralength(200000, 55E-6))
dp = m2h.Cameralength(8, 16.20, '2020-11-20', scale=cl, acceleration_voltage=200000)

In [ ]:
print(dp.as_dataframe())

In [ ]:
mag1 = m2h.Magnification(8000, 16302, '2020-12-01', scale=m2h.Scale(0.34, 'nm'))
mag2 = m2h.Magnification(8000, 16302, '2020-12-01')
cl1 = m2h.Cameralength(8, 16.32, '2020-12-02', scale=m2h.Scale(0.34, '1/nm'), Acceleration_voltage=200000)

df = pd.DataFrame()
df = mag1.add_to_dataframe(df)
df = mag2.add_to_dataframe(df)
df = cl1.add_to_dataframe(df)

print(df)

In [12]:
cl = DiffractionScale(0.000135)
print(cl)
print(cl.to_inv_nm(200000))
print(cl.calculate_cameralength(200000, 55E-6))
dp = m2h.Cameralength(8, 16.20, '2020-11-20', scale=cl, acceleration_voltage=200000)

DiffractionScale Scale=0.000135 1/Å
DiffractionScale Scale=1.35e-05 1/nm
16.24474187833144


In [13]:
print(dp.as_dataframe())

   Nominal Cameralength (cm)  Cameralength (cm)  Scale (1/Å)  Scale (mrad)  \
0                        8.0               16.2     0.000135      0.003386   

   Scale (deg)        Date  Acceleration_voltage  Mode Mag_mode Alpha  Spot  \
0     0.000194  2020-11-20                200000  None     None  None  None   

  Spot_size Camera Microscope  
0      None   None       None  


In [3]:
mag1 = m2h.Magnification(8000, 16302, '2020-12-01', scale=m2h.Scale(0.34, 'nm'))
mag2 = m2h.Magnification(8000, 16302, '2020-12-01')
cl1 = m2h.Cameralength(8, 16.32, '2020-12-02', scale=m2h.Scale(0.34, '1/nm'), Acceleration_voltage=200000)

df = pd.DataFrame()
df = mag1.add_to_dataframe(df)
df = mag2.add_to_dataframe(df)
df = cl1.add_to_dataframe(df)

print(df)

   Nominal Magnification ()  Magnification ()  Scale (nm)        Date  \
0                    8000.0           16302.0        0.34  2020-12-01   
1                    8000.0           16302.0         NaN  2020-12-01   
2                       NaN               NaN         NaN  2020-12-02   

  Acceleration_voltage  Mode Mag_mode Alpha  Spot Spot_size Camera Microscope  \
0                 None  None     None  None  None      None   None       None   
1                 None  None     None  None  None      None   None       None   
2               200000  None     None  None  None      None   None       None   

   Scale (nm/px)  Nominal Cameralength (cm)  Cameralength (cm)  Scale (1/nm)  \
0            NaN                        NaN                NaN           NaN   
1            NaN                        NaN                NaN           NaN   
2            NaN                        8.0              16.32          0.34   

   Scale (mrad)  Scale (deg)  
0           NaN          NaN  

In [3]:
df = mag1.as_dataframe()
df = mag2.add_to_dataframe(df)
print(df)


   Nominal Magnification ()  Magnification ()        Date  \
0                    8000.0           16302.0  2020-12-01   
1                    8000.0           16302.0  2020-12-02   

  Acceleration_voltage  Mode Mag_mode Alpha  Spot Spot_size Camera  
0                 None  None     None  None  None      None   None  
1                 None  None     None  None  None      None   None  


In [4]:
df = mag1.as_dataframe()
df = mag2.add_to_dataframe(df, remove_duplicates=True)
print(df)
#print(mag)

   Nominal {self.name} ({self.units})  {self.name} ({self.units})        Date  \
0                              8000.0                     16302.0  2020-12-02   
1                              8000.0                     16302.0  2020-12-01   

  Acceleration_voltage  Mode Mag_mode Alpha  Spot Spot_size Camera  
0                 None  None     None  None  None      None   None  
1                 None  None     None  None  None      None   None  


In [ ]:
microscope = m2h.Microscope()

print(microscope)

In [2]:
microscope.set_acceleration_voltage(200)
microscope.set_alpha(4)
microscope.set_mode('NBD')
microscope.set_nominal_cameralength(8)
microscope.set_cameralength(16.2)

print(microscope)
print(microscope.get_defined_parameters_(as_dict=True))

Parameter           Value     Units    Nominal value
------------------  --------  -------  ---------------
HT                  200000.0  V
Mode                NBD
Alpha               4
Magnification       nan                nan
Camera length       16.2      cm       8
Magnification Mode
Rocking angle       nan       deg      nan
Rocking frequency   nan       Hz
Step Y              nan       nm       nan
Step X              nan       nm       nan
Convergence angle   nan       mrad     nan
Condenser aperture  nan       um       nan
Spot                nan
Spotsize            nan       nm       nan
Acquisition Date
{'ht': {'Value': 200000.0}, 'mode': {'Value': 'NBD'}, 'alpha': {'Value': 4}, 'camera_length': {'nominal_value': 8, 'actual_value': 16.2}}
